# [Advent of Code 2018](https://adventofcode.com/2018)

## Notes

To get better at Python, I sometimes will refactor the initial solution into something more concise and readable. I will always put the initial soltuion first and put any refactored code below.

## Some helper code

Parts are copied and adapted from [Peter Norvig's code](https://nbviewer.jupyter.org/github/norvig/pytudes/blob/master/ipynb/Advent%202017.ipynb) from 2017.

This will likely grow as the challange goes on.

In [1]:
from itertools import (permutations, combinations, chain, cycle, product, islice, 
                       takewhile, zip_longest, count as count_from)
from collections import (namedtuple, defaultdict)
from datetime import (datetime)
import re

letters  = "abcdefghijklmnopqrstuvwxyz"
BIG = 10 ** 999


def Input(day):
    "Open this day's input file."
    filename = 'data/input{}.txt'.format(day)
    return open(filename)

def Inputstr(day): 
    "The contents of this day's input file as a str."
    return Input(day).read().rstrip('\n')

def first(iterable, default=None): 
    "The first item in an iterable, or default if it is empty."
    return next(iter(iterable), default)

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

def quantify(iterable, pred=bool):
    "Count how many times the predicate is true."
    return sum(map(pred, iterable))

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result
        
################ 2-D points implemented using (x, y) tuples

def X(point): return point[0]
def Y(point): return point[1]

origin = (0, 0)
HEADINGS = UP, LEFT, DOWN, RIGHT = (0, -1), (-1, 0), (0, 1), (1, 0)

def turn_right(heading): return HEADINGS[HEADINGS.index(heading) - 1]
def turn_around(heading):return HEADINGS[HEADINGS.index(heading) - 2]
def turn_left(heading):  return HEADINGS[HEADINGS.index(heading) - 3]

def add(A, B): 
    "Element-wise addition of two n-dimensional vectors."
    return mapt(sum, zip(A, B))

def neighbors4(point): 
    "The four neighboring squares."
    x, y = point
    return (          (x, y-1),
            (x-1, y),           (x+1, y), 
                      (x, y+1))

def neighbors8(point): 
    "The eight neighboring squares."
    x, y = point 
    return ((x-1, y-1), (x, y-1), (x+1, y-1),
            (x-1, y),             (x+1, y),
            (x-1, y+1), (x, y+1), (x+1, y+1))

def cityblock_distance(P, Q=origin): 
    "Manhatten distance between two points."
    return sum(abs(p - q) for p, q in zip(P, Q))

def distance(P, Q=origin): 
    "Straight-line (hypotenuse) distance between two points."
    return sum((p - q) ** 2 for p, q in zip(P, Q)) ** 0.5

def king_distance(P, Q=origin):
    "Number of chess King moves between two points."
    return max(abs(p - q) for p, q in zip(P, Q))


# Day 1

## Data

In [2]:
day1_data = mapt(int, Inputstr(1).splitlines())

## Part One

In [3]:
sum(day1_data)

500

## Part Two

In [4]:
def get_repeated_sequence():
    result = 0
    results = {0}
    while True:
        for val in day1_data:
            result += val
            if result in results:
                return result
            else:
                results.add(result)

get_repeated_sequence()

709

# Day 2

## Data

In [5]:
day2_data = tuple(Inputstr(2).splitlines())

## Part One

In [6]:
twice_count = 0
thrice_count = 0
for id in day2_data:
    has_twice = False
    has_trice = False
    for c in id:
        c_count = id.count(c)
        if c_count == 2:
            has_twice = True
        elif c_count == 3:
            has_trice = True
        if has_twice and has_trice:
            break
    if has_twice:
        twice_count += 1
    if has_trice:
        thrice_count += 1
    
twice_count * thrice_count

6225

### Refactor

In [7]:
def has_char_occur_exact_n_times(id, n):
    return any(id.count(c) == n for c in id)

def has_char_occur_twice(id):
    return has_char_occur_exact_n_times(id, 2)

def has_char_occur_thrice(id):
    return has_char_occur_exact_n_times(id, 3)

twice_count = quantify(day2_data, has_char_occur_twice)
thrice_count = quantify(day2_data, has_char_occur_thrice)
twice_count * thrice_count

6225

## Part Two

In [8]:
def common_chars(id1, id2):
    return ''.join(c1 for (c1, c2) in zip(id1, id2) if c1 == c2)

def get_correct_common_part():
    for (id1, id2) in combinations(day2_data, 2):
        common = common_chars(id1, id2)
        if len(common) + 1 == len(id1):
            return common
        
get_correct_common_part()

'revtaubfniyhsgxdoajwkqilp'

# Day3

## Data

In [9]:
def parse_input_line(line):
    tokens = tuple(line.split(" "))
    # "#"int_str -> int
    id = int(tokens[0][1:])
    # tokens[1] is a @ and not needed
    # int_str","int_str: -> (int, int)
    top_left = mapt(int, tokens[2][:-1].split(","))
    # int_str"x"int_str -> (int, int)
    (width, height) = mapt(int, tokens[3].split("x"))
    return (id, top_left, width, height)
    
day3_data = mapt(parse_input_line, Inputstr(3).splitlines())

## Part One

In [10]:
data = [[0 for x in range(1000)] for x in range(1000)]
for (_, top_left, width, height) in day3_data:
    for x in range(top_left[0], top_left[0]+ width):
        for y in range(top_left[1], top_left[1] + height):
            data[x][y] += 1
count = 0            
for x in range(1000):
    for y in range(1000):
        if data[x][y] > 1:
            count += 1
            
count

110389

## Part Two

In [11]:
def transform_some_more(data):
    (id, top_left, width, height) = data
    bottom_right = (top_left[0] + width - 1, top_left[1] + height - 1)
    return (id, top_left, bottom_right)
    
day3_data_p_two = mapt(transform_some_more, day3_data)

def overlap(box1, box2):
    (_, min1, max1) = box1
    (_, min2, max2) = box2
    if min1[0] > max2[0] or max1[0] < min2[0]:
        return False
    if min1[1] > max2[1] or max1[1] < min2[1]:
        return False
    return True

def do_it():
    for i in range(len(day3_data_p_two)):
        this_is_it = True
        box1 = day3_data_p_two[i]
        for j in range(len(day3_data_p_two)):
            if i == j:
                continue
            box2 = day3_data_p_two[j]
            if overlap(box1, box2):
                this_is_it = False
                break
        if this_is_it:
            return box1[0]
        
do_it()
            

552

# Day 4

## Data

In [12]:
guard_re = re.compile(r"\d+")


def parse_input_line(line):
    # [1518-09-16 23:57] Guard #1889 begins shift
    date_str = line[1:17]
    date = datetime.fromisoformat(date_str)
    rest = line[19:]
    if rest == "falls asleep":
        return (date, 0)
    elif rest == "wakes up":
        return (date, 1)
    else:
        return (date, 2, int(guard_re.search(rest).group()))


day4_data = sorted(
    mapt(parse_input_line,
         Inputstr(4).splitlines()), key=lambda x: x[0])

## Part One

In [13]:
guard_id = None
sleep_minute = None
guard_d = dict()
for datum in day4_data:
    if datum[1] == 0:
        sleep_minute = datum[0].minute
    elif datum[1] == 1:
        for i in range(sleep_minute, datum[0].minute):
            guard_d[guard_id][i] += 1
    else:
        guard_id = datum[2]
        if guard_id not in guard_d:
            guard_d[guard_id] = [0 for x in range(60)]
max_sum = 0
most_slept = None
for id, times in guard_d.items():
    s = sum(times)
    if s > max_sum:
        most_slept = id
        max_sum = s
times = guard_d[most_slept]
max_minute = times.index(max(times))

max_minute * most_slept


76357

## Part Two

In [14]:
biggest_max = 0
minute = None
guard = None
for id, times in guard_d.items():
    m = max(times)
    if m > biggest_max:
        biggest_max = m
        minute = times.index(m)
        guard = id

minute * guard


41668

# Day 5

## Data

In [15]:
day5_data = Inputstr(5)

## Part One

In [16]:
polymer = day5_data

def check(a, b):
    return abs(ord(a) - ord(b)) == 32

def react(polymer):
    buf = []
    for c in polymer:
        if buf and check(c, buf[-1]):
            buf.pop()
        else:
            buf.append(c)
    return len(buf)
    
react(polymer)
  

9238

## Part Two

In [17]:
shortest = BIG
for a,A in zip(letters, letters.upper()):
    polymer = day5_data.replace(a, "").replace(A, "")
    shortest = min(shortest, react(polymer))
    
print(shortest)

4052


# Day 6
## Data

In [18]:
def parse_input_line(line):
    return mapt(int, line.split(", "))

day6_data = mapt(parse_input_line, Inputstr(6).splitlines())

## Part 1

In [19]:
# hmmm, not sure yet

## Part 2

In [20]:
# TBD

# Day 7

## Data

In [21]:
regex = re.compile(r"[A-Z]")

def parse_input_line(line):
    # Step Z must be finished before step V can begin.
    return tuple(reversed(regex.findall(line)[1:]))

day7_data = mapt(parse_input_line, Inputstr(7).splitlines())

## Part One

In [22]:
D = dict()
for c in letters.upper():
    D[c] = list()
for k, v in day7_data:
    D[k].append(v)
result = []
while len(D) > 0:
    step = sorted([k for k,v in D.items() if len(v) == 0])[0]
    result.append(step)
    D.pop(step)
    for _, v in D.items():
        try:
            v.remove(step)
        except ValueError:
            pass
"".join(result)

'JMQZELVYXTIGPHFNSOADKWBRUC'

## Part Two

In [23]:
def req_t(step):
    return ord(step) - ord("A") + 61

D = dict()
for c in letters.upper():
    D[c] = list()
for k, v in day7_data:
    D[k].append(v)
    
t = 0
idle_workers = [0,1,2,3,4]
active_workers = []
while True:
    # process active workers
    if active_workers:
        done_workers = [(w, s) for (w, s, s_t) in active_workers if req_t(s) == s_t]
        active_workers = [(w, s, s_t + 1) for (w, s, s_t) in active_workers if req_t(s) > s_t]
        # add done workers back to idle pile and update step deps
        for (worker, step) in done_workers:
            idle_workers.append(worker)
            for _, v in D.items():
                try:
                    v.remove(step)
                except ValueError:
                    pass
    # process idle workers
    if idle_workers:
        available_steps = sorted([k for k,v in D.items() if len(v) == 0])
        min_s = min(len(idle_workers), len(available_steps))
        for _ in range(min_s):
            worker = idle_workers.pop(0)
            step = available_steps.pop(0)
            D.pop(step)
            # steps get one unit from here
            active_workers.append((worker, step, 1))
    # check if we are done
    # we musn't increment t on the tick we're done to get the correct result
    if not D and not active_workers:
        break
    t += 1
    
t

1133

# Day 8

## Data

In [24]:
day8_data = list(map(int, Inputstr(8).split(" ")))

## Part One

In [25]:
metadata = []
i = 0
def walk():
    global metadata
    global i
    num_childs = day8_data[i]
    num_metadata = day8_data[i + 1]
    i += 2
    for _ in range(num_childs):
        walk()
    metadata.extend(day8_data[i : i + num_metadata])
    i += num_metadata
walk()
sum(metadata)

40908

## Part Two

In [27]:
i = 0
child_map = defaultdict(dict)
node_count = 1
def walk(parent, child):
    global i
    global child_map
    global node_count
    me = node_count
    node_count += 1
    num_childs = day8_data[i]
    num_metadata = day8_data[i + 1]
    i += 2
    for c in range(1, num_childs + 1):
        walk(me, c)
    metadata = day8_data[i : i + num_metadata]
    i += num_metadata
    if num_childs == 0:
        child_map[parent][child] = sum(metadata)
    else:
        child_map[parent][child] = sum([child_map[me].get(m, 0) for m in metadata])
walk(0, 1)
child_map[0][1]
        

25910